In [1]:
#load in dependencies
import pandas as pd
import numpy as np
import bz2
import json

In [2]:
blacklist = ['Fluxbox', 'fluxbox', 'ps2pdf', 'cps3', 'mps3', 'Trixbox', 'rps25', 'aps2', 'gps2', 'lps3', 'pps25', r'a-zA-Z'+'ps', r'a-zA-Z'+'xbox']
ps_list = ['playstation', 'ps1', 'ps2', 'ps3', 'ps4']
xbox_list = ['xbox']
#-------------------------------should probably run mention defs before cleaning data
def ps_mention(row):
    mention = False
    if not any(entry in row for entry in blacklist):
        if any(x in row for x in ps_list):
            mention = True
    return mention

def xbox_mention(row):
    mention = False
    if not any(entry in row for entry in blacklist):
        if any(x in row for x in xbox_list):
            mention = True
    return mention

def cleaning(uncleaned_df):
    cleaned = uncleaned_df[['id', 'score', 'controversiality', 'subreddit', 'body', 'month', 'year']]
    original_size = len(uncleaned_df.index)
    cleaned['original_size'] = original_size
    cleaned['PS'] = cleaned['body'].apply(ps_mention)
    cleaned['XBOX'] = cleaned['body'].apply(xbox_mention)
    #print(cleaned.shape)#still a normal sized file at this point
    df = cleaned[(cleaned['PS'] == True) | (cleaned['XBOX'] == True)]
    #print(df.shape)#df size is incorrect or there are few mentions in early years
    df.set_index('id', inplace=True, drop=True) 
    return(df)

In [3]:
#years = ['2006', '2007', '2008', '2009']
years = ['2009'] #for testing
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for y in years:
    for m in months:
        df = pd.DataFrame()
        try:
            filename = "data/" + y + "/RC_" + y + "-" + m + ".bz2"
            parsed_file = pd.read_json(filename, compression='bz2', lines=True)
            parsed_file['month'] = m
            parsed_file['year'] = y
            cleaned_monthly = cleaning(parsed_file)
            df = df.append(cleaned_monthly)
            print(filename + " parsed")
            df.to_json('dataframes/df'+y+'-'+m+'.json')
            
        except:
            print("file named " + filename + " does not exist")
    print(df.shape)
    #df.to_json('dataframes/df'+y+'.json')
    print('saved dataframes/df'+y)

<ipython-input-2-426fea0ba100>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['original_size'] = original_size
<ipython-input-2-426fea0ba100>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned['PS'] = cleaned['body'].apply(ps_mention)
<ipython-input-2-426fea0ba100>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs

data/2009/RC_2009-01.bz2 parsed
data/2009/RC_2009-02.bz2 parsed
data/2009/RC_2009-03.bz2 parsed
data/2009/RC_2009-04.bz2 parsed
data/2009/RC_2009-05.bz2 parsed
data/2009/RC_2009-06.bz2 parsed
data/2009/RC_2009-07.bz2 parsed
data/2009/RC_2009-08.bz2 parsed
data/2009/RC_2009-09.bz2 parsed
data/2009/RC_2009-10.bz2 parsed
data/2009/RC_2009-11.bz2 parsed
data/2009/RC_2009-12.bz2 parsed
(2096, 9)
saved dataframes/df2009


In [7]:
compiled_df = pd.DataFrame()
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
for m in months:
    url = 'dataframes/df2009-'+m+'.json'
    df = pd.read_json(url)
    compiled_df = compiled_df.append(df)
    compiled_df.to_json('dataframes/df2009.json')
print(compiled_df.shape)

(11545, 9)
